In [326]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_validate,KFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

In [327]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [328]:
set_train = pd.read_csv('data_set.csv')
test_values = pd.read_csv('data_test.csv', index_col='building_id')

In [329]:
columnas_seleccionada = ['age',
                         'geo_level_1_id',
                         'area_percentage',
                         'count_floors_pre_eq',
                         'geo_level_2_id',
                         'height_percentage',
                         'resistance',
                         'roof_type',
                         'ground_floor_type',
                         'other_floor_type',
                         'position_j', 'position_o', 
                         'position_s', 'position_t',
                         'base_condition']

In [330]:
columnas_seleccionadas = ['age',
                         'geo_level_1_id',
                         'area_percentage',
                         'count_floors_pre_eq',
                         'geo_level_2_id',
                         'height_percentage',
                         'resistance',
                         'roof_type',
                         'ground_floor_type',
                         'other_floor_type',
                         'position_j', 'position_o', 
                         'position_s', 'position_t',
                        #'owner_a', 'owner_r',
                         #'owner_v', 'owner_w',
                         'base_condition']

In [331]:
train, test = train_test_split(set_train, test_size = 0.20, shuffle = False)

In [332]:
train_values_subset = train.iloc[:, 0:-1]
train_labels = train.loc[:, 'damage_grade']

In [333]:
test_values_subset = test.iloc[:, 0:-1]
test_labels = test.loc[:, 'damage_grade']

In [334]:
# A parameter grid for XGBoost
#n_jobs=[-1]
#n_estimators=np.arange(100,1000,100)
#learning_rate=[0.03,0.01,0.1]
#max_depth=np.arange(10,100,15)
# Param grid for Xgboost
#param_grid={'n_jobs':n_jobs,
 #           'n_estimators':n_estimators,
  #          'max_depth':max_depth,
   #         'learning_rate':learning_rate
    #       }

In [335]:
#rf=XGBClassifier()
#kf=KFold(n_splits=2,shuffle=True)

In [336]:
#rs=RandomizedSearchCV(rf,param_distributions=param_grid,cv=kf,scoring='f1_micro')

In [337]:
#random_search = RandomizedSearchCV(rf_model, param_distributions=params, n_iter=500, scoring='accuracy', n_jobs=1, cv=skf.split(train_values_subset,train_labels.values.ravel()), random_state=1)

In [338]:
rf_model = XGBClassifier(n_jobs=-1,n_estimators= 400, max_depth= 10,learning_rate= 0.1, eval_metric='mlogloss', colsample_bytree = 0.9)
# n_estimators 400 -> 0.7264 SUBMIT - 0.7336697881486031
# n_estimators 400, colsample_bytree = 0.9 -> 0.7358381946576604
# n_estimators 400, colsample_bytree = 0.8 -> 0.7353968375805956
# n_estimators 400, colsample_bytree = 0.7 -> 0.7354544058949954


In [339]:
train_values_subset = pd.get_dummies(train_values_subset)

In [ ]:
start_time = timer(None)
rf_model.fit(train_values_subset, train_labels.values.ravel())
timer(start_time) 

In [ ]:
# *----- Submit actual 0.7257
preds = rf_model.predict(test_values_subset)
f1_score(test_labels, preds, average='micro')

In [ ]:
#rs.best_params_ 0.7332284310715382
#rs.best_params_ 

In [ ]:
test_values_subset = test_values[columnas_seleccionadas]
test_values_subset = pd.get_dummies(test_values_subset)
test_values_subset

In [ ]:
predictions = rf_model.predict(test_values_subset)

In [ ]:
submission_format = pd.read_csv('submission_format.csv',index_col='building_id')

In [ ]:
my_submission = pd.DataFrame(data=predictions, columns=submission_format.columns,
                            index=submission_format.index)

In [ ]:
my_submission.head()

In [ ]:
my_submission.to_csv('submision.csv')

In [ ]:
!head submision.csv

In [ ]:
rf_model.feature_importances_

In [ ]:
# Gini importance
rf_model.feature_importances_
columnas_seleccionadas = ['age',
                         'geo_level_1_id',
                         'area_percentage',
                         'count_floors_pre_eq',
                         'geo_level_2_id',
                         'height_percentage',
                         'resistance',
                         'roof_type',
                         'ground_floor_type',
                         'other_floor_type',
                    #     'position_j', 'position_o', 
                     #    'position_s', 'position_t',
                         'owner_a', 'owner_r',
                         'owner_v', 'owner_w',
                         'base_condition']

In [ ]:
plt.bar(test_values_subset.columns, rf_model.feature_importances_)
plt.xlabel('Features')
plt.ylabel('Importancia')
plt.title('Importancia Features con RF')
plt.show()